# NLP CW Task 1

#### Running your code:
  - Your models should run automatically when running your colab file without further intervention
  - For each task you should automatically output the performance of both models
  - Your code should automatically download any libraries required

#### Structure of your code:
  - You are expected to use the 'train', 'eval' and 'model_performance' functions, although you may edit these as required
  - Otherwise there are no restrictions on what you can do in your code

#### Documentation:
  - You are expected to produce a .README file summarising how you have approached both tasks
  - Your .README file should explain how to replicate the different experiments mentioned in your report

# Setup

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split
from collections import Counter
import nltk
import string
import torchtext
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LinearRegression, Ridge
from tqdm import tqdm
import re
import spacy

In [ ]:
# Get pretrained GloVe embeddings
glove = torchtext.vocab.GloVe(name='6B', dim=50)

In [ ]:
# Install packages that Colab does not provide automatically
!pip -q install transformers
from transformers import RobertaTokenizer, RobertaModel, BertPreTrainedModel, DistilBertModel, DistilBertTokenizer

# Download stopwords
nltk.download('stopwords')

# Download spacy model
!spacy download en_core_web_sm -q
ner = spacy.load('en_core_web_sm')

# Get test and train data files
if not os.path.exists('dev.csv'):
  !wget -q --show-progress https://raw.githubusercontent.com/matt-malarkey/nlp-cw-data/master/dev.csv
  !wget -q --show-progress https://raw.githubusercontent.com/matt-malarkey/nlp-cw-data/master/train.csv
  !wget -q --show-progress https://raw.githubusercontent.com/matt-malarkey/nlp-cw-data/master/competition_test.csv

In [ ]:
# Set random seed and device
SEED = 1
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('dev.csv')
comp_df = pd.read_csv('competition_test.csv')

In [ ]:
# Build columns in dataset for edited headline and original word
def build_edit_cols(dataset):
  edited = []
  original_words = []
  for i in range(len(dataset)):
    original_words.append(re.search(r'<(.*)/>', dataset['original'][i]).group(1))
    edited.append(re.sub(r'<.*/>', dataset['edit'][i], dataset['original'][i]))
  dataset['edited'] = edited
  dataset['originalWord'] = original_words

build_edit_cols(train_df)
build_edit_cols(test_df)
build_edit_cols(comp_df)

# Approach 1
Using pre-trained representations

In [ ]:
# Training loop
def train(train_iter, dev_iter, model, number_epoch, optimizer):
  loss_fn = nn.MSELoss()
  loss_fn = loss_fn.to(DEVICE)

  performance_stats = []

  for epoch in range(1, number_epoch+1):
    model.train()

    epoch_loss = 0
    epoch_sse = 0
    no_observations = 0  # Observations used for training so far

    for batch in train_iter:
      ids = batch['ids'].to(DEVICE).squeeze()
      mask = batch['mask'].to(DEVICE).squeeze()
      target = batch['target'].to(DEVICE, dtype=torch.float)
      extra_features = batch['extra_features'].to(DEVICE)

      predictions = model(ids, mask, extra_features).squeeze(1)
      optimizer.zero_grad()
      loss = loss_fn(predictions, target)

      no_observations = no_observations + target.shape[0]
      sse, __ = model_performance(predictions.detach().cpu().numpy(), target.detach().cpu().numpy())

      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()*target.shape[0]
      epoch_sse += sse

    valid_loss, valid_mse, __, __ = eval(dev_iter, model)

    epoch_loss, epoch_mse = epoch_loss / no_observations, epoch_sse / no_observations
    performance_stats.append((valid_loss, valid_mse**0.5))
    print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train MSE: {epoch_mse:.2f} | Train RMSE: {epoch_mse**0.5:.2f} | \
    Val. Loss: {valid_loss:.2f} | Val. MSE: {valid_mse:.2f} |  Val. RMSE: {valid_mse**0.5:.2f} |')

  return performance_stats

In [ ]:
# Evaluate performance
def eval(data_iter, model):
  model.eval()
  epoch_loss = 0
  epoch_sse = 0
  pred_all = []
  trg_all = []
  no_observations = 0
  loss_fn = nn.MSELoss()
  loss_fn = loss_fn.to(DEVICE)

  with torch.no_grad():
    for batch in data_iter:
      ids = batch['ids'].to(DEVICE).squeeze()
      mask = batch['mask'].to(DEVICE).squeeze()
      target = batch['target'].to(DEVICE, dtype=torch.float)
      extra_features = batch['extra_features'].to(DEVICE)

      predictions = model(ids, mask, extra_features).squeeze(1)

      loss = loss_fn(predictions, target)

      # We get the mse
      pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
      sse, __ = model_performance(pred, trg)

      no_observations = no_observations + target.shape[0]
      epoch_loss += loss.item()*target.shape[0]
      epoch_sse += sse
      pred_all.extend(pred)
      trg_all.extend(trg)

  return epoch_loss/no_observations, epoch_sse/no_observations, np.array(pred_all), np.array(trg_all)

In [ ]:
# Get SSE and MSE
def model_performance(output, target, print_output=False):
  sq_error = (output - target)**2
  sse = np.sum(sq_error)
  mse = np.mean(sq_error)
  rmse = np.sqrt(mse)

  if print_output:
    print(f'| MSE: {mse:.2f} | RMSE: {rmse:.2f} |')

  return sse, mse

# Data Analysis


In [ ]:
# Basic Stats
num_headlines = len(train_df)
max_len = max(train_df['original'].apply(len))
avg_score = train_df['meanGrade'].mean()
print(f'Number of Headlines: {num_headlines}, Max Headline Length: {max_len}, Avg Score: {avg_score:.2f}')

# Most common words
stopwords = nltk.corpus.stopwords.words('english')
most_common_words = Counter(" ".join(train_df['original']).split()).most_common(100)
stopwords.append(string.punctuation)
most_common_words = [(word, count) for (word, count) in most_common_words if word not in stopwords and word.isalpha()][:5]
print(f'Most common words: {most_common_words}')

# Analysis of Trump Headlines
trump_headlines = train_df[train_df['original'].str.lower().str.contains('trump')]
num_headlines_with_trump = len(trump_headlines)
trump_avg_score = trump_headlines['meanGrade'].mean()
prob_headline_contains_trump = num_headlines_with_trump / num_headlines
trump_value_counts = trump_headlines['meanGrade'].value_counts(bins=3)
trump_proportion_gt_one = (trump_value_counts[2] + trump_value_counts[3]) * 100 / num_headlines_with_trump
print(f'Headlines with Trump: {prob_headline_contains_trump * 100:.2f}%, Avg Score: {trump_avg_score:.2f}')
print(f'Proportion of Trump Headlines with score > 1: {trump_proportion_gt_one:.2f}%')

# Analysis of word similarity and length of headlines
semantic_similarity_data = []
sentence_similarity_data = []
levenshtein_similarity_data = []
headline_length_data = []

for row in train_df.itertuples():
  all_words = row.original.split(' ')
  headline_length_data.append((len(all_words), row.meanGrade))
  sentence_vec = torch.zeros((1,50))
  original_word_vec = glove[row.originalWord.lower()].unsqueeze(0)
  edited_word_vec = glove[row.edit.lower()].unsqueeze(0)

  for i, word in enumerate(all_words):
    if len(word) > 2 and word[0] == '<' and word[-2] == '/' and word[-1] == '>' :
      word = word[1:-2].lower()
    sentence_vec += glove[word.lower()].unsqueeze(0)

  semantic_distance = torch.cosine_similarity(original_word_vec, edited_word_vec).item()
  levenshtein_distance = nltk.edit_distance(row.originalWord.lower(), row.edit.lower())
  semantic_similarity_data.append((semantic_distance, row.meanGrade))
  levenshtein_similarity_data.append((levenshtein_distance, row.meanGrade))
  sentence_semantic_distance = torch.cosine_similarity(sentence_vec, edited_word_vec)
  sentence_similarity_data.append((sentence_semantic_distance, row.meanGrade))


# Analysis of # of clause separators
separators_data = []
for i in range(len(train_df['original'])):
  matches = re.findall("[;,:(...)]", train_df['original'][i])
  separators_data.append((len(matches), train_df['meanGrade'][i]))

# Plot word similarity vs meanGrade
plt.figure(figsize=(15, 10))
plt.subplot(2, 3, 1)
mean_stat_similarity = binned_statistic(*zip(*semantic_similarity_data), bins=6, range=(-0.5, 1.0))
plt.plot(mean_stat_similarity.bin_edges[:-1] + 0.125, mean_stat_similarity.statistic)
plt.xlabel('Cosine Distance')
plt.ylabel('Avg Score')
plt.title('Semantic similarity vs meanGrade')

# Levenshtein distance vs meanGrade
plt.subplot(2, 3, 2)
mean_stat_similarity = binned_statistic(*zip(*levenshtein_similarity_data), bins=6)
plt.plot(mean_stat_similarity.bin_edges[:-1] + 0.125, mean_stat_similarity.statistic)
plt.xlabel('Levenshtein Distance')
plt.ylabel('Avg Score')
plt.title('Word spelling similarity vs meanGrade')

# Semantic similarity of word to headline vs meanGrade
plt.subplot(2, 3, 3)
mean_stat_similarity = binned_statistic(*zip(*sentence_similarity_data), bins=6)
plt.plot(mean_stat_similarity.bin_edges[:-1] + 0.125, mean_stat_similarity.statistic)
plt.xlabel('Cosine Distance')
plt.ylabel('Avg Score')
plt.title('Semantic similarity of word to headline vs meanGrade')

# Plot clause separator count vs meanGrade
plt.subplot(2, 3, 4)
plt.plot()
mean_stat_seps = binned_statistic(*zip(*separators_data), bins=6)
plt.plot(mean_stat_seps.bin_edges[:-1], mean_stat_seps.statistic)
plt.xlabel('Number of Clause Separators')
plt.ylabel('Avg Score')
plt.title('Clause Separator Count vs meanGrade')

# Plot headline length vs meanGrade
plt.subplot(2, 3, 5)
mean_stat_headline_len = binned_statistic(*zip(*headline_length_data), bins=30, range=(0, 30))
plt.plot(mean_stat_headline_len.bin_edges[:-1], mean_stat_headline_len.statistic)
plt.xlabel('Number of Words')
plt.ylabel('Avg Score')
plt.title('Headline Length vs meanGrade')

plt.show()

## Preprocessing

In [ ]:
# Pattern to match words and multiword tags
# See https://regex101.com/r/a2cWZL/1 for example
# TOKEN_PATTERN = re.compile("[A-Za-z#]+|<[A-Za-z#]+\s[A-Za-z]+\/>")

# Patten to match only words
TOKEN_PATTERN = re.compile("\w+")
CLAUSE_SEPS = re.compile("[;,:(...)]")

# TODO: processing of:
# - hashtags,
# - words with a "-" in between them
# - punctuation
# - entity names

def tokenize(sentence):
  return TOKEN_PATTERN.findall(sentence)

# Remove angle brackets and lowercase
def preprocess(word):
  if len(word) > 2 and word[0] == '<' and word[-2] == '/' and word[-1] == '>' :
    word = word[1:-2]
  return word.lower()

def create_vocab(data):
  """
  Creating a corpus of all the tokens used
  """
  # Let us put the tokenized corpus in a list
  tokenized_corpus = [[preprocess(t) for t in tokenize(s)] for s in data]

  # Create single list of all vocabulary
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
      if token not in vocabulary:
        vocabulary.append(token)

  return vocabulary, tokenized_corpus

# Extra features:
# [0] - binary if 'Trump' in headline
# [1] - length of headline
# [2] - number of clauses in a headline
# [3] - semantic distance between original and edit word glove embedding
# [4] - semantic distance between edit word and total original headline glove embedding
# [5] - edit distance between edit word and original word
EXTRA_FEATURES = 6
def extract_features(raw_headline, original_word, edit_word):
  sentence_vec = torch.zeros((1, 50))
  original_word_vec = glove[original_word.lower()].unsqueeze(0)
  edited_word_vec = glove[edit_word.lower()].unsqueeze(0)

  for i, word in enumerate(raw_headline.split(' ')):
    if len(word) > 2 and word[0] == '<' and word[-2] == '/' and word[-1] == '>' :
      word = word[1:-2].lower()
    sentence_vec += glove[word.lower()].unsqueeze(0)

  semantic_distance = torch.cosine_similarity(original_word_vec, edited_word_vec).item()
  levenshtein_distance = nltk.edit_distance(original_word.lower(), edit_word.lower())
  sentence_semantic_distance = torch.cosine_similarity(sentence_vec, edited_word_vec)

  return [
    int('Trump' in raw_headline),
    len(raw_headline),
    len(CLAUSE_SEPS.findall(raw_headline)) + 1,
    semantic_distance,
    sentence_semantic_distance,
    levenshtein_distance
  ]

EXTRA_FEATURES_TWO = 3
def extract_features_two(raw_headline):
  return [
    int('Trump' in raw_headline),
    len(raw_headline),
    len(CLAUSE_SEPS.findall(raw_headline)) + 1
  ]

In [ ]:
# Inherits from Dataset class so the DataLoader can use it
class Task1Dataset(Dataset):

  def __init__(self, tokenizer, train_data, labels):
    self.x_train = train_data
    self.y_train = labels
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.y_train)

  def __getitem__(self, item):
    # Preprocess headline before embedding
    headline = self.x_train[0][item]
    original_word = self.x_train[1][item]
    edit_word = self.x_train[2][item]
    # processed_headline = ' '.join([preprocess(t) for t in tokenize(headline)])
    processed_headline = [preprocess(t) for t in tokenize(headline)]

    target = self.y_train[item]

    # Use is_split_into_words=True to allow for our own tokenizing
    # - stops encode_plus splitting up unknown words into multiple parts
    encoding = self.tokenizer.encode_plus(processed_headline, return_tensors='pt',
                                          padding='max_length', truncation=True,
                                          max_length=128, pad_to_max_length=True,
                                          is_split_into_words=True)
    ids = encoding['input_ids']
    mask = encoding['attention_mask']

    extra_features = extract_features(headline, original_word, edit_word)

    return {
      'ids': torch.tensor(ids),
      'mask': torch.tensor(mask),
      'target': torch.tensor(target),
      'extra_features': torch.tensor(extra_features)
    }

In [ ]:
class BertRegressionModel(nn.Module):
  def __init__(self, model_name):
    super(BertRegressionModel, self).__init__()

    if model_name == 'distilbert':
      self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
    elif model_name == 'roberta':
      self.bert = RobertaModel.from_pretrained('roberta-base')

    self.num_extra_features = EXTRA_FEATURES
    self.linear1 = nn.Linear(768, 32)
    self.linear2 = nn.Linear(32 + self.num_extra_features, 1)

  def forward(self, ids, mask, extra_features):
    embeddings = self.bert(input_ids=ids, attention_mask=mask)[0]
    x = self.linear1(embeddings[:, 0])
    concat_features = torch.cat([x, extra_features], dim=1)
    x = self.linear2(concat_features)
    return x

### Approach 1: Using pre-trained representations

In [ ]:
# Number of epochs
epochs = 10

# Proportion of training data for train compared to dev
train_proportion = 0.8

BATCH_SIZE = 32

# Initialise chosen BERT model and tokenizer
# To change the model to use Roberta, pass in 'roberta' to the regression model
# and uncomment the tokenizer below.
model = BertRegressionModel('distilbert').to(DEVICE)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

training_data = [train_df['edited'], train_df['originalWord'], train_df['edit']]
training_labels = train_df['meanGrade']
test_data = test_df['edited']
train_and_dev = Task1Dataset(tokenizer, training_data, training_labels)

# Split training data into train and dev sets
train_examples = round(len(train_and_dev)*train_proportion)
dev_examples = len(train_and_dev) - train_examples
train_dataset, dev_dataset = random_split(train_and_dev, (train_examples, dev_examples))

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE)

In [ ]:
# Do the training
distil_model = BertRegressionModel('distilbert').to(DEVICE)
optimizer_distil = torch.optim.Adam(distil_model.parameters())
tokenizer_distil = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

roberta_model = BertRegressionModel('roberta').to(DEVICE)
optimizer_roberta = torch.optim.Adam(roberta_model.parameters())
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')

print('Training roberta model...')
performance_stats_roberta = train(train_loader, dev_loader, roberta_model, epochs, optimizer_roberta)
print('Training distil-bert model...')
performance_stats_distil = train(train_loader, dev_loader, distil_model, epochs, optimizer_distil)

In [ ]:
# Save models
torch.save(roberta_model, 'roberta-model.pt')
torch.save(distil_model, 'distilbert-model.pt')

In [ ]:
# Plot losses for both pre trained models
epoch_losses_1, rmse_1 = zip(*performance_stats_distil)
epoch_losses_2, rmse_2 = zip(*performance_stats_roberta)
x = np.arange(epochs) + 1

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(x, epoch_losses_1, label='distil-bert')
plt.plot(x, epoch_losses_2, label='roberta')
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Pre-trained model losses')

plt.subplot(1, 2, 2)
plt.plot(x, rmse_1, label='distil-bert')
plt.plot(x, rmse_2, label='roberta')
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Pre-trained model RMSE')
plt.show()

**Approach 1: Evaluation**

In [ ]:
def comp_eval():
  model = roberta_model
  # Evaluate model on the competition data
  tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

  # Set training and test data
  comp_ids = comp_df['id']
  comp_data = [comp_df['edited'], comp_df['originalWord'], comp_df['edit']]
  comp_labels = comp_df['meanGrade']

  comp_data = Task1Dataset(tokenizer, comp_data, comp_labels)
  comp_loader = torch.utils.data.DataLoader(comp_data, batch_size=32)

  _,_,predictions,_ = eval(comp_loader, model)

  comp_preds = {'id': comp_ids, 'pred': predictions}
  comp_preds = pd.DataFrame(comp_preds)
  comp_preds.to_csv('pred_outputs.csv', index=False)
  print('Saved pred_outputs.csv\n')

  min_score = min(predictions)
  max_score = max(predictions)
  mean_score = sum(predictions) / len(predictions)
  range = max_score - min_score
  print(f'Min predicted score: {min_score}, Max predicted score: {max_score}, Range: {range}, Mean Predicted Score: {mean_score}\n')  
  print(f'MSE and RMSE on the Competition Test Set:')
  output = model_performance(predictions, comp_labels, print_output=True)
comp_eval()

## Approach 2
No pre-trained representations

In [ ]:
# TODO: comment on what this is

train_and_dev = train_df['edited']
training_data, dev_data, training_y, dev_y = train_test_split(train_df['edited'], train_df['meanGrade'],
                                                                        test_size=(0.2),
                                                                        random_state=42)

print(len(training_data))

# We train a Tf-idf model
#count_vect = CountVectorizer(stop_words='english')
#count_vect = CountVectorizer(stop_words='english', ngram_range=(2,2))
count_vect = CountVectorizer(stop_words='english', ngram_range=(3,3))
train_counts = count_vect.fit_transform(training_data)
#transformer = TfidfTransformer().fit(train_counts)
#train_counts = transformer.transform(train_counts)
#regression_model = LinearRegression().fit(train_counts, training_y)
regression_model = Ridge().fit(train_counts, training_y)

train_sizes=[1, 100, 500, 1000, 2000, 5000, 6176]

train_sizes, train_scores, validation_scores = learning_curve(
    estimator = LinearRegression(),
    X = train_counts,
    y = training_y, train_sizes = train_sizes, cv = 5,
    scoring = 'neg_mean_squared_error')

train_scores_mean = -train_scores.mean(axis = 1)
validation_scores_mean = -validation_scores.mean(axis = 1)
print(train_scores)

plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training error')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
plt.ylabel('MSE', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.legend()

# Train predictions
predicted_train = regression_model.predict(train_counts)

# Calculate Tf-idf using train and dev, and validate model on dev:
test_and_test_counts = count_vect.transform(train_and_dev)
#transformer = TfidfTransformer().fit(test_and_test_counts)

test_counts = count_vect.transform(dev_data)

#test_counts = transformer.transform(test_counts)

# Dev predictions
predicted = regression_model.predict(test_counts)

# We run the evaluation:
print("\nTrain performance:")
sse, mse = model_performance(predicted_train, training_y, True)

print("\nDev performance:")
sse, mse = model_performance(predicted, dev_y, True)

#### Baseline for task 2

In [ ]:
# Baseline for the task
pred_baseline = torch.zeros(len(dev_y)) + np.mean(training_y)
print("\nBaseline performance:")
sse, mse = model_performance(pred_baseline, dev_y, True)

### Neural Network Approach

In [ ]:
# Preprocessing steps for recognising entities, building vocab

# Tokenize with Spacy Named Entity Recognition
ENTITY_TOKEN_PATTERN = re.compile('<.+?>|\w+')
def tokenize_headline_with_ner(headline):
  # Entity recognition - wrap entities in angle brackets
  doc = ner(headline)
  for e in doc.ents:
    headline = re.sub(e.text, f"<{e.text}>", headline)

  # Tokenize headline (respecting detected entities)
  headline = [t.lower() for t in ENTITY_TOKEN_PATTERN.findall(headline)]

  return headline

# Create training vocab, with dedicated words for Spacy recognised entities
def create_vocab_with_ner(data, stopwords):
  vocabulary = []
  max_len = 0
  for headline in data:
    headline = tokenize_headline_with_ner(headline)

    # Keep track of longest headline
    if len(headline) > max_len:
      max_len = len(headline)
    
    # Add new tokens to vocab
    for token in headline:
      if token not in vocabulary:
        vocabulary.append(token)

  # Convert vocab list to dict
  w2i = {w: idx+1 for (idx, w) in enumerate(vocabulary)}
  w2i['<pad>'] = 0

  return w2i, max_len

In [122]:
class Task1DatasetNN(Dataset):

  def __init__(self, train_data, labels, max_len, word2idx):
    self.x_train = train_data
    self.y_train = labels
    self.max_len = max_len
    self.word2idx = word2idx

  def __len__(self):
    return len(self.y_train)

  def __getitem__(self, item):
    headline = self.x_train[item]

    # Get extra features before processing
    extra_features = torch.FloatTensor(extract_features_two(headline))

    # Get tokenized headline, map to words/entities in vocab
    headline = tokenize_headline_with_ner(headline)
    vectorized_headline = [self.word2idx[t] for t in headline if t in self.word2idx]

    # Build headline tensor with 0 padding if needed
    headline_tensor = torch.zeros((self.max_len,)).long()
    headline_tensor[:len(vectorized_headline)] = torch.LongTensor(vectorized_headline[:self.max_len])

    target_tensor = torch.FloatTensor([self.y_train[item]])

    return {
      'encoding': headline_tensor,
      'extra_features': extra_features,
      'target': target_tensor
    }

#### Feed Forward Neural Network

In [ ]:
class FFNN(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, vocab_size, num_extra_features):
    super(FFNN, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
    self.fc1 = nn.Linear(embedding_dim + num_extra_features, hidden_dim)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(hidden_dim, 1)

  # Average sentence embedding (ignoring padding)
  # Maps embedded : (batch_size, max_sent_len, embedding_dim) => (batch_size, embeddind_dim)
  def average_embedding(self, embedded):
    total_emb = torch.sum(embedded, dim=1)
    non_zero_embs = torch.sum(embedded != 0, dim=[1])
    return total_emb / non_zero_embs

  def forward(self, x, extra_features):
    # x has shape (batch_size, max_sent_len)
    # extra_features has shape (batch_size, num_extra_features)

    # Learn embedding
    embedded = self.embedding(x)
    averaged = self.average_embedding(embedded)

    # Concat extra features
    out = torch.cat([averaged, extra_features], dim=1)
    
    # Hidden and output layers
    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    
    return out

In [ ]:
# Train model
def train_nn(train_loader, model, loss_fn, optimizer):
  loss_fn = loss_fn.to(DEVICE)
  performance_stats = []

  for epoch in range(EPOCHS):
    model.train()

    epoch_loss = 0
    epoch_sse = 0
    no_observations = 0 

    for batch in train_loader:

      encoding = batch['encoding'].to(DEVICE)
      target = batch['target'].to(DEVICE).squeeze(1)
      extra_features = batch['extra_features'].to(DEVICE)

      optimizer.zero_grad()
      predictions = model(encoding, extra_features).squeeze(1)

      # Compute the loss
      loss = loss_fn(predictions, target)
      train_loss = loss.item()

      # sse stats
      no_observations = no_observations + target.shape[0]
      sse, __ = model_performance(predictions.detach().cpu().numpy(), target.detach().cpu().numpy())

      loss.backward()
      optimizer.step()

      epoch_loss += train_loss * target.shape[0]
      epoch_sse += sse

    valid_loss, valid_mse, __, __ = eval_nn(dev_loader, model, loss_fn)

    epoch_loss, epoch_mse = epoch_loss / no_observations, epoch_sse / no_observations
    performance_stats.append((valid_loss, valid_mse**0.5))
    print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train MSE: {epoch_mse:.2f} | Train RMSE: {epoch_mse**0.5:.2f} | \
Val. Loss: {valid_loss:.2f} | Val. MSE: {valid_mse:.2f} |  Val. RMSE: {valid_mse**0.5:.2f} | Train Loss (Single?): {train_loss:.3f}')

In [ ]:
# Evaluate performance
def eval_nn(data_iter, model, loss_fn):
  model.eval()
  epoch_loss = 0
  epoch_sse = 0
  pred_all = []
  trg_all = []
  no_observations = 0
  loss_fn = loss_fn.to(DEVICE)

  with torch.no_grad():
    for batch in data_iter:

      encoding = batch['encoding'].to(DEVICE)
      target = batch['target'].to(DEVICE).squeeze(1)
      extra_features = batch['extra_features'].to(DEVICE)

      predictions = model(encoding, extra_features).squeeze(1)
      loss = loss_fn(predictions, target)

      # Get the mse
      pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
      sse, __ = model_performance(pred, trg)

      no_observations = no_observations + target.shape[0]
      epoch_loss += loss.item()*target.shape[0]
      epoch_sse += sse
      pred_all.extend(pred)
      trg_all.extend(trg)

  return epoch_loss/no_observations, epoch_sse/no_observations, np.array(pred_all), np.array(trg_all)

In [ ]:
EPOCHS = 10
LRATE = 0.01
EMBEDDING_DIM = 10 # ~4th root of vocab size
HIDDEN_DIM = 50
OUTPUT_DIM = 1
BATCH_SIZE = 50
train_proportion = 0.8

# Set training data
training_data = train_df['edited']
training_labels = train_df['meanGrade']

# Build vocab with NER
stopwords = nltk.corpus.stopwords.words('english')
w2i, max_headline_len = create_vocab_with_ner(training_data, stopwords)

train_and_dev = Task1DatasetNN(training_data, training_labels, max_headline_len, w2i)

# Split training data into train and dev sets
train_examples = round(len(training_data)*train_proportion)
dev_examples = len(training_data) - train_examples
train_dataset, dev_dataset = random_split(train_and_dev, (train_examples, dev_examples))

# Create dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE)

In [119]:
# Construct the model
model = FFNN(EMBEDDING_DIM, HIDDEN_DIM, vocab_size=len(w2i), num_extra_features=EXTRA_FEATURES_TWO)
model = model.to(DEVICE)
print(model)

# Use MSE loss and the Adam optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LRATE)

train_nn(train_loader, model, loss_fn, optimizer)
eval_nn(dev_loader, model, loss_fn)

FFNN(
  (embedding): Embedding(12077, 10, padding_idx=0)
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)
| Epoch: 00 | Train Loss: 0.55 | Train MSE: 0.55 | Train RMSE: 0.74 | Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.60 | Train Loss (Single?): 0.246
| Epoch: 01 | Train Loss: 0.34 | Train MSE: 0.34 | Train RMSE: 0.58 | Val. Loss: 0.39 | Val. MSE: 0.39 |  Val. RMSE: 0.62 | Train Loss (Single?): 0.312
| Epoch: 02 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 | Val. Loss: 0.40 | Val. MSE: 0.40 |  Val. RMSE: 0.63 | Train Loss (Single?): 0.353
| Epoch: 03 | Train Loss: 0.23 | Train MSE: 0.23 | Train RMSE: 0.48 | Val. Loss: 0.45 | Val. MSE: 0.45 |  Val. RMSE: 0.67 | Train Loss (Single?): 0.110
| Epoch: 04 | Train Loss: 0.18 | Train MSE: 0.18 | Train RMSE: 0.42 | Val. Loss: 0.40 | Val. MSE: 0.40 |  Val. RMSE: 0.64 | Train Loss (Single?): 0.204
| Epoch: 05 | Train Loss: 0.15 | Train MSE: 

(0.474731114244214,
 0.474731107820501,
 array([ 0.95104885,  1.0807132 , -0.22058079, ...,  1.2073691 ,
         1.3676199 ,  0.7415808 ], dtype=float32),
 array([1.6, 2. , 0.6, ..., 0. , 0.8, 0. ], dtype=float32))

#### Approach 2: Evaluation

In [126]:
def comp_eval2():
  # Set training and test data
  comp_ids = comp_df['id']
  comp_labels = comp_df['meanGrade']

  comp_data = Task1DatasetNN(comp_df['edited'], comp_labels, max_headline_len, w2i)
  comp_loader = torch.utils.data.DataLoader(comp_data, batch_size=BATCH_SIZE)

  _,_,predictions,_ = eval_nn(comp_loader, model, loss_fn)

  comp_preds = {'id': comp_ids, 'pred': predictions}
  comp_preds = pd.DataFrame(comp_preds)
  comp_preds.to_csv('pred_outputs_approach2.csv', index=False)
  print('Saved pred_outputs2.csv\n')
  # predictions = [x if str(x) != "nan" else 0 for x in predictions]

  min_score = min(predictions)
  max_score = max(predictions)
  mean_score = sum(predictions) / len(predictions)
  range = max_score - min_score
  print(f'Min predicted score: {min_score}, Max predicted score: {max_score}, Range: {range}, Mean Predicted Score: {mean_score}\n')  
  print(f'MSE and RMSE on the Competition Test Set:')
  output = model_performance(predictions, comp_labels, print_output=True)
comp_eval2()

Saved pred_outputs2.csv

Min predicted score: -1.8207062482833862, Max predicted score: 4.194506645202637, Range: 6.0152130126953125, Mean Predicted Score: nan

MSE and RMSE on the Competition Test Set:
| MSE: 0.59 | RMSE: 0.77 |
